# Notes

## Flac Format

https://xiph.org/flac/format.html

## Flac C API

https://xiph.org/flac/api/modules.html

https://xiph.org/flac/api/structFLAC____StreamMetadata.html

Source files: https://git.xiph.org/?p=flac.git;a=tree;f=src/libFLAC;h=c172637587ca9bfe4cd4156955db3dbcf5a42395;hb=refs/heads/master

## Flac command line tool and library files

Command line: `flac %s --endian=little --channels=%d --sample-rate=%d --bps=%d --sign=signed -%d -o %s`

ie. `flac 3544749_0005.dat --endian=little --channels=4 --sample-rate=125 --bps=8 --sign=signed -0`

Example flac file: `/home/cx1111/Downloads/data/mimic3wdb/matched/p000020/3544749_0005.flac`

Show shared libraries of an executable: `ldd /usr/bin/flac`

Show callable functions from a library: `nm -D /usr/lib/x86_64-linux-gnu/libFLAC.so.8`

nm -D `/lib/x86_64-linux-gnu/libc-2.23.so`

## Calling C functions in Python

Options
- ctypes (built in): https://docs.python.org/3/library/ctypes.html
- python/c api (built in): https://docs.python.org/3.5/c-api/
- swig: https://github.com/dsully/pyflac
- boost

http://book.pythontips.com/en/latest/python_c_extension.html
https://stackoverflow.com/questions/145270/calling-c-c-from-python


## Building C Extensions

https://docs.python.org/3/extending/building.html


# Compiling c

gcc -c -fPIC simple.c -o simple.o
gcc -shared simple.o -o simple.so


In [1]:
import ctypes
simple = ctypes.CDLL('./compress/simple.so')

In [3]:
class Singer(ctypes.Structure):
    _fields_ = [
                ('name', ctypes.c_char * 20),
                ('networth', ctypes.c_int),
    ]
    
simple.setsinger.argtypes = [ctypes.c_char_p, ctypes.c_int, ctypes.POINTER(Singer)]
simple.setsinger.restype = None

s = Singer()

name = b'Sean'
simple.setsinger(name, 100000, ctypes.byref(s))

print(s.name.decode('ascii'))
print(s.networth)

Sean
100000


In [1]:
import ctypes

flac = ctypes.CDLL('/usr/lib/x86_64-linux-gnu/libFLAC.so.8')

class FLAC__StreamMetadata_StreamInfo(ctypes.Structure):
    _fields_ = [('min_blocksize', ctypes.c_uint),
                ('max_blocksize', ctypes.c_uint),
                ('min_framesize', ctypes.c_uint),
                ('max_framesize', ctypes.c_uint),
                ('sample_rate', ctypes.c_uint),
                ('channels', ctypes.c_uint),
                ('bits_per_sample', ctypes.c_uint),
                ('total_samples', ctypes.c_ulonglong),
                ('md5sum', ctypes.c_char * 16),
               ]

# Specify input and output types
flac.FLAC__metadata_get_streaminfo.argtype = [ctypes.c_char_p, ctypes.POINTER(FLAC__StreamMetadata_StreamInfo)]
flac.FLAC__metadata_get_streaminfo.restype = ctypes.c_bool
    
meta = FLAC__StreamMetadata_StreamInfo()
x = flac.FLAC__metadata_get_streaminfo(b'/home/cx1111/Downloads/data/mimic3wdb/matched/p000020/3544749_0005.flac',
                                       ctypes.byref(meta))

In [5]:
meta.sample_rate

1152

In [6]:
libc = ctypes.CDLL('/lib/x86_64-linux-gnu/libc-2.23.so')
libc.printf(b"Hello, %s\n", b"World!")

14

In [5]:
libc.time(None)

1527191722

In [1]:
import os
import subprocess
import time 

import cxutils as cx
from wfdb.io import compress_file, test_compression, compare_compressions, rdheader
from wfdb.io._signal import wfdbfmtres


In [2]:
fmts = ['bz2']* 3 + ['gzip']* 3 + ['lz4']*3 + ['zstd']*3 + ['flac']*3
compress_levels = [1, 5, 9,
                   1, 5, 9,
                   0, 10, 16,
                   1, 15, 22,
                   0, 5, 8]

compression_results, dataset_info = compare_compressions(fmts, compress_levels)

print('\n\nBenchmark results of %d files, with total size %s' % (dataset_info['n_files'], dataset_info['uncompressed_total']))
display(compression_results)

Testing bz2 with compress level=1 ...
Testing bz2 with compress level=5 ...
Testing bz2 with compress level=9 ...
Testing gzip with compress level=1 ...
Testing gzip with compress level=5 ...
Testing gzip with compress level=9 ...
Testing lz4 with compress level=0 ...
Testing lz4 with compress level=10 ...
Testing lz4 with compress level=16 ...
Testing zstd with compress level=1 ...
Testing zstd with compress level=15 ...
Testing zstd with compress level=22 ...
Testing flac with compress level=0 ...
Testing flac with compress level=5 ...
Testing flac with compress level=8 ...
Full benchmark complete


Benchmark results of 5980 files, with total size 21.01 G


,fmt,compress_level,compression_ratio,time_compress,time_decompress
0,bz2,1,2.01,0:41:48,0:16:54
1,bz2,5,2.14,0:52:01,0:22:37
2,bz2,9,2.18,1:11:38,0:44:29
3,gzip,1,1.60,0:10:08,0:03:45
4,gzip,5,1.69,0:19:50,0:03:36
5,gzip,9,1.70,0:37:15,0:03:43
6,lz4,0,1.17,0:02:47,0:02:12
7,lz4,10,1.48,0:23:51,0:00:32
8,lz4,16,1.48,0:44:57,0:00:30
9,zstd,1,1.57,0:02:33,0:01:17


In [ ]:
fmts = ['bz2', 'bz2', 'bz2']
compress_levels = [1, 5, 9]

compression_results, dataset_info = compare_compressions(fmts, compress_levels)

print('\n\nBenchmark results of %d files, with total size %s' % (dataset_info['n_files'], dataset_info['uncompressed_total']))
display(compression_results)

In [ ]:
fmts = ['gzip', 'gzip', 'gzip']
compress_levels = [1, 5, 9]

compression_results, dataset_info = compare_compressions(fmts, compress_levels)

print('\n\nBenchmark results of %d files, with total size %s' % (dataset_info['n_files'], dataset_info['uncompressed_total']))
display(compression_results)

In [ ]:
fmts = ['lz4', 'lz4']
compress_levels = [0, 16]

compression_results, dataset_info = compare_compressions(fmts, compress_levels)

print('\n\nBenchmark results of %d files, with total size %s' % (dataset_info['n_files'], dataset_info['uncompressed_total']))
display(compression_results)

In [ ]:
fmts = ['zstd', 'zstd', 'zstd']
compress_levels = [1, 15, 22]

compression_results, dataset_info = compare_compressions(fmts, compress_levels)

print('\n\nBenchmark results of %d files, with total size %s' % (dataset_info['n_files'], dataset_info['uncompressed_total']))
display(compression_results)

In [4]:
fmts = ['flac', 'flac', 'flac']
compress_levels = [0, 5, 8]

compression_results, dataset_info = compare_compressions(fmts, compress_levels)

print('\n\nBenchmark results of %d files, with total size %s' % (dataset_info['n_files'], dataset_info['uncompressed_total']))
display(compression_results)

Testing flac with compress level=0 ...
Full benchmark complete


Benchmark results of 5980 files, with total size 21.01 G


,fmt,compress_level,compression_ratio,time_compress,time_decompress
0,flac,0,2.18,360.514716,297.462226
